## Mean Reversion EURUSD Bot

### Imports and Data Download

In [67]:
import pandas as pd
import numpy as np
import yfinance as yf
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleCollector, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
from config import access_token, accountID
import json
from oandapyV20.endpoints.accounts import AccountSummary
from tabulate import tabulate

data = yf.download('EURUSD=X', start='2000-01-01')
data = data[['Close']]
data = data.dropna()

# half_life = 263*4*24  # From mean-reversion-and-stationarity.ipynb
half_life = 263

[*********************100%%**********************]  1 of 1 completed


### Calculate Strategy Parameters

In [68]:
data['MovingAvg'] = data['Close'].rolling(window=half_life).mean()
data['MovingStd'] = data['Close'].rolling(window=half_life).std()
data['Z'] = (data['Close'] - data['MovingAvg']) / data['MovingStd']
data['Position'] = -data['Z']
data['PnL'] = data['Position'].shift(1) * (data['Close'].diff())
data['Cumulative PnL'] = data['PnL'].cumsum()

In [69]:
data[data['MovingAvg'].notna()]

,Close,MovingAvg,MovingStd,Z,Position,PnL,Cumulative PnL
Date,,,,,,,
2004-12-02,1.326102,1.235336,0.031802,2.854127,-2.854127,NaN,NaN
2004-12-03,1.346602,1.235906,0.032442,3.412051,-3.412051,-0.058510,-0.058510
2004-12-06,1.339603,1.236403,0.033023,3.125065,-3.125065,0.023881,-0.034629
2004-12-07,1.343093,1.236901,0.033638,3.156935,-3.156935,-0.010908,-0.045537
2004-12-08,1.334508,1.237381,0.034124,2.846251,-2.846251,0.027104,-0.018433
...,...,...,...,...,...,...,...
2024-02-06,1.074183,1.082829,0.016261,-0.531663,0.531663,-0.001156,0.982496
2024-02-07,1.075720,1.082772,0.016260,-0.433724,0.433724,0.000817,0.983313
2024-02-08,1.077575,1.082767,0.016262,-0.319278,0.319278,0.000804,0.984117


### Trading Job

In [70]:
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M1)
    candles = collector.grab(n)
    return candles

In [76]:
lst = []

api = API(access_token=access_token)
response = AccountSummary(accountID=accountID)    
api.request(response)

{'account': {'guaranteedStopLossOrderMode': 'DISABLED',
  'hedgingEnabled': False,
  'id': '101-001-28220236-001',
  'createdTime': '2024-02-04T15:03:12.467843741Z',
  'currency': 'USD',
  'createdByUserID': 28220236,
  'alias': 'Primary',
  'marginRate': '0.02',
  'lastTransactionID': '19234',
  'balance': '99989.9509',
  'openTradeCount': 3,
  'openPositionCount': 1,
  'pendingOrderCount': 4,
  'pl': '-9.5063',
  'resettablePL': '-9.5063',
  'resettablePLTime': '0',
  'financing': '-0.5428',
  'commission': '0.0000',
  'dividendAdjustment': '0',
  'guaranteedExecutionFees': '0.0000',
  'unrealizedPL': '-1.1459',
  'NAV': '99988.8050',
  'marginUsed': '34.4128',
  'marginAvailable': '99954.3922',
  'positionValue': '1720.6397',
  'marginCloseoutUnrealizedPL': '-1.0221',
  'marginCloseoutNAV': '99988.9288',
  'marginCloseoutMarginUsed': '34.4128',
  'marginCloseoutPositionValue': '1720.6397',
  'marginCloseoutPercent': '0.00017',
  'withdrawalLimit': '99954.3922',
  'marginCallMarginUs

In [84]:
def trading_job():
    # Fetching latest price data
    candles = get_candles(half_life) 
    dfstream = pd.DataFrame(columns=['Open', 'Close', 'High', "Low"])

    # Parsing candle data for trading signals
    for i, candle in enumerate(candles):
        dfstream.loc[i] = [
            float(candle.bid.o.value),  
            float(candle.bid.c.value),
            float(candle.bid.h.value),
            float(candle.bid.l.value)
        ]

    # Calculate the rolling mean and standard deviation for the z-score
    dfstream['MovingAvg'] = dfstream['Close'].rolling(window=half_life).mean()  
    dfstream['MovingStd'] = dfstream['Close'].rolling(window=half_life).std()
    dfstream['Z'] = (dfstream['Close'] - dfstream['MovingAvg']) / dfstream['MovingStd']

    # Dropping the initial rows where the rolling window is not full
    # dfstream.dropna(inplace=True)

    # Generate signals based on the z-score
    dfstream['Signal'] = 0  # Initialize all signals to 0
    dfstream.loc[dfstream['Z'] > 1, 'Signal'] = 1   # Sell signal
    dfstream.loc[dfstream['Z'] < -1, 'Signal'] = 2  # Buy signal

    last_signal = dfstream['Signal'].iloc[-1]

    client = API(access_token)

    print(tabulate(dfstream.tail(), headers='keys', tablefmt='psql'))

    SLTPRatio = 2
    previous_candleR = abs(dfstream['Open'].iloc[-2] - dfstream['Close'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    positionDF = pd.DataFrame({"TPBuy": TPBuy, "SLBuy": SLBuy, "TPSell": TPSell, "SLSell": SLSell}, index=[0])
    print(tabulate(positionDF, headers='keys', tablefmt='psql'))
    
    account_summary = response.response.get("account")
    total_profit_loss = account_summary.get("balance")

    print(f"Total Profit/Loss: {json.dumps(total_profit_loss, indent=2)}")
    lst.extend(total_profit_loss)

    # Sell
    if last_signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(json.dumps(rv, indent=2))

    # Buy
    elif last_signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(json.dumps(rv, indent=2))

### Executing Trades Manually

In [85]:
# MANUAL
trading_job()

# SCHEDULED
scheduler = BlockingScheduler()
# scheduler.add_job(trading_job, 'cron', day_of_week='mon-sun', hour='00-23', minute='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59', second='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59', start_date='2023-09-26 01:00:00')
scheduler.add_job(trading_job, 'cron', day_of_week='mon-sun', hour='00-23', minute='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59', second='9,28,47')
scheduler.start()

+-----+---------+---------+---------+---------+-------------+---------------+------------+----------+
|     |    Open |   Close |    High |     Low |   MovingAvg |     MovingStd |          Z |   Signal |
|-----+---------+---------+---------+---------+-------------+---------------+------------+----------|
| 258 | 1.07721 | 1.07736 | 1.07738 | 1.07721 |   nan       | nan           | nan        |        0 |
| 259 | 1.07735 | 1.07738 | 1.07742 | 1.07732 |   nan       | nan           | nan        |        0 |
| 260 | 1.07738 | 1.07738 | 1.0774  | 1.07735 |   nan       | nan           | nan        |        0 |
| 261 | 1.07736 | 1.07737 | 1.07738 | 1.07731 |   nan       | nan           | nan        |        0 |
| 262 | 1.07738 | 1.0774  | 1.07742 | 1.07738 |     1.07768 |   0.000420167 |  -0.669931 |        0 |
+-----+---------+---------+---------+---------+-------------+---------------+------------+----------+
+----+---------+---------+----------+----------+
|    |   TPBuy |   SLBuy |   TPSe